In [153]:
import os ,re
import time, datetime
import requests 
from requests_html import HTML 
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from pathlib import Path
import pandas as pd

In [154]:
BASE_DIR = Path.cwd()
DATA_DIR = BASE_DIR / "data"  #os.path.join(BASE_DIR, 'data')
if not DATA_DIR.exists():
    DATA_DIR.mkdir(exist_ok=True) #os.mkdirs(DATA_DIR, exists_ok=TRUE)

    
product_category_links_ouput = DATA_DIR / "category-products.csv"
product_output = DATA_DIR / "products.csv"

In [155]:
cwd = os.getcwd() # dirname of abs(path(__file__))
BASE_DIR = os.path.dirname(cwd)
BASE_DIR

'/home/vianney/projects/data-science-projects/notebooks'

In [156]:
options= Options()
options.add_argument(" --headless")
driver = webdriver.Chrome(options=options)

In [157]:
# dict or all url
categories = [
    { "name":"toys-and-games", "url": "https://www.amazon.com/Best-Sellers-Toys-Games/zgbs/toys-and-games/"},
    {"name":"electronis", "url":"https://www.amazon.com/Best-Sellers-Electronics/zgbs/electronics/"},
    {"name":"fashion", "url":"https://www.amazon.com/Best-Sellers/zgbs/fashion/"},
    {"name":"Gifts-cards", "url":"https://www.amazon.com/Best-Sellers-Gift-Cards/zgbs/gift-cards/"},
    
]
    

In [158]:
regex_options = [
    r"https://www.amazon.com/gp/product/(?P<product_id>[\w-]+)/",
    r"https://www.amazon.com/dp/(?P<product_id>[\w-]+)/",
    r"https://www.amazon.com/(?P<slug>[\w-]+)/dp/(?P<product_id>[\w-]+)/",
    r"https://www.amazon.com/gp/product/(?P<product_id>[\w-]+)/",
]

def extract_product_id_from_url(url):
    product_id = None
    for regex_str in regex_options:
        regex = re.compile(regex_str)
        match = regex.match(url)
        if match != None:
            try:
                product_id = match['product_id']
            except:
                pass
    return product_id

In [159]:
#page_links = [ x for x in page_links if extract_product_id_from_url(x) != None]
def clean_page_links(page_links=[], category = None):
    final_page_links = []
    for url in page_links:
        product_id = extract_product_id_from_url(url) 
        if product_id != None :
            final_page_links.append({"url": url, "product_id": product_id, "category":category})
    return final_page_links

In [160]:
def scrape_category_product_links(categories=[]):
    all_product_links = []
    for category in categories:
        time.sleep(1.5)
        url = category.get("url")
        driver.get(url)
        body_el = driver.find_element(By.CSS_SELECTOR,"body")
        html_str = body_el.get_attribute("innerHTML")
        html_obj = HTML(html=html_str)
        page_links = [f"https://www.amazon.com{x}" for x in  html_obj.links if x.startswith("/")]
        cleaned_links =  clean_page_links(page_links=page_links, category=category)
        all_product_links += cleaned_links
    return  all_product_links  

all_product_links  = scrape_category_product_links(categories)

In [161]:
all_product_links

[{'url': 'https://www.amazon.com/dp/B09NY8P1S3/ref=zg_bs_toys-and-games_11/147-1102269-1485954?pd_rd_i=B09CHMJMS6&psc=1',
  'product_id': 'B09NY8P1S3',
  'category': {'name': 'toys-and-games',
   'url': 'https://www.amazon.com/Best-Sellers-Toys-Games/zgbs/toys-and-games/'}}]

In [162]:
#Put product categories in dataframe and save
def extract_categories_and_save(categories=categories):
    all_product_links  = scrape_category_product_links(categories)
    category_df = pd.DataFrame(all_product_links )
    category_df.to_csv(product_category_links_ouput, index=False)

In [163]:
extract_categories_and_save(categories=categories)

In [164]:
def scrape_product_page(url):
    driver.get(url)
    time.sleep(0.3)
    body_el = driver.find_element(By.CSS_SELECTOR,"body")
    html_str = body_el.get_attribute("innerHTML")
    html_obj = HTML(html=html_str)
    product_title = driver.find_element(By.ID,'productTitle').text
    product_price = driver.find_element(By.XPATH,"//span[@class='a-price a-text-price a-size-medium apexPriceToPay']").text
    return product_title , product_price

In [165]:
def perform_scrape(cleaned_items=[]):
    data_extracted = []
    for obj in cleaned_items:
        link= obj['url']
        product_id = obj['product_id']
        title, price = (None, None)
        try:
            title, price = scrape_product_page(link)
        except:
            pass
        if title != None and price != None:
            #print(link, title, price)
            product_data = {
                "url": link,
                "product_id": product_id,
                "title": title,
                "price": price            

            }
        data_extracted.append(product_data)
        
    return data_extracted


In [115]:
#extracted_data = perform_scrape(cleaned_items=cleaned_links)

In [39]:
print(extracted_data)

In [166]:
def row_scrape_event(row, *args, **kwargs):
    link = row['url']
    scraped = 0
    try:
        scraped = row['scraped']
    except:
        pass
    # print(link)
    if scraped == 1 or scraped == "1":
        print("skipped")
        return row
    product_id = row['product_id']
    title, price = (None, None)
    try:
        title, price = scrape_product_page(link)
    except:
        pass
    row['title'] = title
    row['price'] = price
    row['scraped'] = 1
    row['timestamp'] = datetime.datetime.now().timestamp()
    print(link, title, price)
    return row

In [167]:
def  row_scrape_event(row, *args, **kwargs):
    link= row['url']
    scraped = 0   # Make it  work with csv file
    try:
        scraped = row['scraped']
    except:
            pass
    if scraped== 1 or scraped == "1":
        return row
    product_id = row['product_id']
    title, price = (None, None)
    try:
        title, price = scrape_product_page(link)
    except:
        pass
    row['title'] = title
    row['price'] = price
    row['scraped'] = 1
    row['timestamp'] = datetime.datetime.now().timestamp()     
    return row


In [168]:
df = pd.read_csv(product_category_links_ouput )

In [169]:
df.head()

,url,product_id,category
0,https://www.amazon.com/dp/B09NY8P1S3/ref=zg_bs...,B09NY8P1S3,"{'name': 'toys-and-games', 'url': 'https://www..."


In [ ]:
newDf=df.apply(row_scrape_event, axis=1)
newDf

https://www.amazon.com/Flybar-Jumper-Toddlers-Durable-Supports/dp/B00WTDXSDM/ref=zg_bs_toys-and-games_6/144-0817804-9574927?pd_rd_i=B0916L8NWZ&psc=1 Flybar My First Foam Pogo Jumper for Kids Fun and Safe Pogo Stick for Toddlers, Durable Foam and Bungee Jumper for Ages 3 and up, Supports up to 250lbs $16.99
https://www.amazon.com/Hasbro-A5640-Connect-4-Game/dp/B00D8STBHY/ref=zg_bs_toys-and-games_19/144-0817804-9574927?pd_rd_i=B00D8STBHY&psc=1 None None
https://www.amazon.com/Mini-Drones-Kids-Adults-Controlled/dp/B07CVFWVF5/ref=zg_bs_toys-and-games_17/144-0817804-9574927?pd_rd_i=B09488QMWN&psc=1 Force1 Scoot Hand Operated Drone for Kids or Adults - Hands Free Motion Sensor Mini Drone, Easy Indoor Small UFO Toy Flying Ball Drone Toy for Boys and Girls (Blue) $26.95
https://www.amazon.com/VTech-Pull-and-Sing-Puppy/dp/B01MQ3YP7Y/ref=zg_bs_toys-and-games_12/144-0817804-9574927?pd_rd_i=B09NT21ZSB&psc=1 VTech Pull and Sing Puppy $16.88
https://www.amazon.com/Rock-Painting-Kit-Kids-Activities/d

,url,product_id,category,title,price,scraped,timestamp
0,https://www.amazon.com/Flybar-Jumper-Toddlers-...,B00WTDXSDM,"{'name': 'toys-and-games', 'url': 'https://www...",Flybar My First Foam Pogo Jumper for Kids Fun ...,$16.99,1,1.640083e+09
1,https://www.amazon.com/Hasbro-A5640-Connect-4-...,B00D8STBHY,"{'name': 'toys-and-games', 'url': 'https://www...",None,None,1,1.640083e+09
2,https://www.amazon.com/Mini-Drones-Kids-Adults...,B07CVFWVF5,"{'name': 'toys-and-games', 'url': 'https://www...",Force1 Scoot Hand Operated Drone for Kids or A...,$26.95,1,1.640083e+09
3,https://www.amazon.com/VTech-Pull-and-Sing-Pup...,B01MQ3YP7Y,"{'name': 'toys-and-games', 'url': 'https://www...",VTech Pull and Sing Puppy,$16.88,1,1.640083e+09
4,https://www.amazon.com/Rock-Painting-Kit-Kids-...,B07Z2R7S5P,"{'name': 'toys-and-games', 'url': 'https://www...",Rock Painting Kit for Kids - Arts and Crafts f...,$14.99,1,1.640083e+09
...,...,...,...,...,...,...,...
86,https://www.amazon.com/Layering-Initial-Neckla...,B08L8Y43C4,"{'name': 'fashion', 'url': 'https://www.amazon...",None,None,1,1.640084e+09
87,https://www.amazon.com/Fruit-Loom-Sweatshirts-...,B09DTFLCYS,"{'name': 'fashion', 'url': 'https://www.amazon...",None,None,1,1.640084e+09
88,https://www.amazon.com/Gildan-Fleece-Hooded-Sw...,B01M1CX371,"{'name': 'fashion', 'url': 'https://www.amazon...",None,None,1,1.640084e+09
89,https://www.amazon.com/followme-45902-9-S-Flee...,B06Y1V56FC,"{'name': 'fashion', 'url': 'https://www.amazon...",None,None,1,1.640084e+09


In [170]:
newDf.to_csv(product_output, index=False)

In [171]:
product_df =pd.read_csv(product_output)
product_df.head()

,url,product_id,category,title,price,scraped,timestamp
0,https://www.amazon.com/Flybar-Jumper-Toddlers-...,B00WTDXSDM,"{'name': 'toys-and-games', 'url': 'https://www...",Flybar My First Foam Pogo Jumper for Kids Fun ...,$16.99,1,1.640083e+09
1,https://www.amazon.com/Hasbro-A5640-Connect-4-...,B00D8STBHY,"{'name': 'toys-and-games', 'url': 'https://www...",NaN,NaN,1,1.640083e+09
2,https://www.amazon.com/Mini-Drones-Kids-Adults...,B07CVFWVF5,"{'name': 'toys-and-games', 'url': 'https://www...",Force1 Scoot Hand Operated Drone for Kids or A...,$26.95,1,1.640083e+09
3,https://www.amazon.com/VTech-Pull-and-Sing-Pup...,B01MQ3YP7Y,"{'name': 'toys-and-games', 'url': 'https://www...",VTech Pull and Sing Puppy,$16.88,1,1.640083e+09
4,https://www.amazon.com/Rock-Painting-Kit-Kids-...,B07Z2R7S5P,"{'name': 'toys-and-games', 'url': 'https://www...",Rock Painting Kit for Kids - Arts and Crafts f...,$14.99,1,1.640083e+09


In [172]:
final_df= pd.concat([product_df,df])
final_df.tail(50)

,url,product_id,category,title,price,scraped,timestamp
42,https://www.amazon.com/Roku-Streaming-Device-V...,B09BKCDXZC,"{'name': 'electronis', 'url': 'https://www.ama...",Roku Streaming Stick 4K 2021 | Streaming Devic...,$39.00,1.0,1.640083e+09
43,https://www.amazon.com/fire-tv-stick-4k-max-wi...,B08MQZXN1X,"{'name': 'electronis', 'url': 'https://www.ama...",NaN,NaN,1.0,1.640083e+09
44,https://www.amazon.com/Fitbit-Fitness-Smartwat...,B07TVC2KLW,"{'name': 'electronis', 'url': 'https://www.ama...",Fitbit Versa 2 Health and Fitness Smartwatch w...,$99.00,1.0,1.640083e+09
45,https://www.amazon.com/Roku-Streaming-Stick-HD...,B075XLWML4,"{'name': 'electronis', 'url': 'https://www.ama...",Roku Streaming Stick+ | HD/4K/HDR Streaming De...,$29.99,1.0,1.640083e+09
46,https://www.amazon.com/Tile-RE-19001-Mate-1-Pa...,B07W9BBCTB,"{'name': 'electronis', 'url': 'https://www.ama...","Tile Mate (2020) 1-pack - Bluetooth Tracker, K...",$17.99,1.0,1.640083e+09
47,https://www.amazon.com/Echo-Dot-3rd-Gen-Blueto...,B08F8RQ469,"{'name': 'electronis', 'url': 'https://www.ama...",Echo Dot (3rd Gen) | Charcoal with Sengled Blu...,$19.99,1.0,1.640083e+09
48,https://www.amazon.com/Amazon-smart-plug-works...,B089DR29T6,"{'name': 'electronis', 'url': 'https://www.ama...",NaN,NaN,1.0,1.640083e+09
49,https://www.amazon.com/Echo-Designed-parental-...,B084J4MJCK,"{'name': 'electronis', 'url': 'https://www.ama...",NaN,NaN,1.0,1.640083e+09
50,https://www.amazon.com/Apple-Watch-Aluminum-Ab...,B09HDXS1DM,"{'name': 'electronis', 'url': 'https://www.ama...",NaN,NaN,1.0,1.640083e+09
51,https://www.amazon.com/Fire-7-Kids-Edition-Tab...,B07H936BZT,"{'name': 'electronis', 'url': 'https://www.ama...",NaN,NaN,1.0,1.640083e+09


Save the data in csv format

In [88]:
final_df= pd.concat([product_df,df])

final_df.to_csv(product_output, index=False)